In [ ]:
%store -r

%set_env MLFLOW_TRACKING_URI={mlflow_arn}
%set_env MLFLOW_EXPERIMENT_NAME=anomaly_detection
%set_env MLFLOW_RUN_NAME=training-kmeans

In [ ]:
# import ModelBuilder from sagemaker
from steps.training_kmeans import train_kmeans
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.spec.inference_spec import InferenceSpec
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.serve.mode.function_pointers import Mode
import sagemaker
import pickle as pkl
import numpy as np

In [ ]:
model, labels = train_kmeans(X_train, 3)

#pickle model to disk
with open('model.pkl', 'wb') as f:
    pkl.dump(model, f)

# upload model to s3
sagemaker_session = sagemaker.Session()
model_s3_path = sagemaker_session.upload_data(path='model.pkl', bucket=sagemaker_session.default_bucket(), key_prefix='models')

In [ ]:
from sagemaker.sklearn.model import SKLearnModel

sklearn_model = SKLearnModel(
    model_data=model_s3_path,
    role=sagemaker.get_execution_role(),
    source_dir='./src/',
    py_version='py3',
    framework_version='1.2-1',
    entry_point='inference.py'
)

In [ ]:

predictor = sklearn_model.deploy(
    instance_type='local',
    initial_instance_count=1,
)

In [ ]:
clusters, dists = predictor.predict(X_test[0:1000].values)

In [ ]:
print(np.where(dists > 16))
np.where(y_test[0:1000].values == 1)